In [2]:
!pip install jcopdl
!pip install gdown 

  Created wheel for jcopdl: filename=jcopdl-1.1.9-py2.py3-none-any.whl size=16925 sha256=dfee0f000221f74ce525690fda1ddd3f9b7a2ce5e5153687da50b316ff66f8ba
  Stored in directory: /root/.cache/pip/wheels/bb/fd/9d/fc72bcf569000eabd487e654fdf55b587e66882b60eaa5693b
Successfully built jcopdl


In [5]:
# Download Datasets
!unrar x "/content/drive/My Drive/datasets/mnist_train.rar" "/content/"

Streaming output truncated to the last 5000 lines.
Extracting  /content/mnist_train/6/img_550.jpg                            88%  OK 
Extracting  /content/mnist_train/1/img_5500.jpg                           88%  OK 
Extracting  /content/mnist_train/6/img_5501.jpg                           88%  OK 
Extracting  /content/mnist_train/1/img_5502.jpg                           88%  OK 
Extracting  /content/mnist_train/4/img_5503.jpg                           88%  OK 
Extracting  /content/mnist_train/6/img_5504.jpg                           88%  OK 
Extracting  /content/mnist_train/9/img_5505.jpg                           88%  OK 
Extracting  /content/mnist_train/5/img_5506.jpg                           88%  OK 
Extracting  /content/mnist_train/0/img_5507.jpg                           88%  OK 
Extracting  /content/mnist_train/6/img_5508.jpg                           88%  OK 
Extracting  /content/mnist_tr

In [6]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Datasets dan DataLoader (Hanya Data Train)

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from jcopdl.layers import linear_block
from torch import nn

In [8]:
bs = 64

data_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # normalize supaya menjadi -1 sampai 1, supaya lebih stabil
])

train_set = datasets.ImageFolder("/content/mnist_train", transform=data_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=8)

In [9]:
image, label = next(iter(trainloader))
image.shape

torch.Size([64, 1, 28, 28])

In [10]:
# set config
config = set_config({
    "z_dim" : 100,
    "bs" :64
})

## Training Preparation

In [28]:
from model_gan_2 import Discriminator, Generator

In [29]:
D = Discriminator().to(device)
G = Generator(config.z_dim).to(device)

criterion = nn.BCELoss()

d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

## Fase Training

In [30]:
import os

from torchvision.utils import save_image

In [31]:
os.makedirs("output/GAN/", exist_ok=True)
os.makedirs("model/GAN/", exist_ok=True)

In [32]:
num_epochs = 200

for epoch in range(num_epochs):
    D.train()
    G.train()
    
    for real_img, _ in trainloader:
        n_data = real_img.shape[0]
        
        # Handle Real image dan Fake image
        real_img = real_img.to(device)
        fake_img = G.generator(n_data, device)
        
        # Handle Real label dan Fake label
        real = torch.ones((n_data, 1), device=device)
        fake = torch.zeros((n_data, 1), device=device)
        
        # Training Discriminator
        d_optimizer.zero_grad()
        ## 1. Real image -> Discriminator -> label real
        output = D(real_img)
        d_real_loss = criterion(output, real)
        
        ## 2. Fake image -> Discriminator -> label fake
        output = D(fake_img.detach())
        d_fake_loss = criterion(output, fake)
        
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Training Generator
        g_optimizer.zero_grad()
        ## Fake image -> Discriminator -> label real -> jangan di detach()
        output = D(fake_img)
        g_loss = criterion(output, real)
        g_loss.backward()
        g_optimizer.step()
        
        
    if epoch % 5 == 0:
        print(f"Epoch: {epoch:5} | D_loss: {d_loss/2:.5f} | G_loss: {g_loss/2:.5f}")
        
    if epoch % 15 == 0:
        G.eval()
        epoch = str(epoch).zfill(4)
        fake_img = G.generator(64, device)
        save_image(fake_img.view(-1, 1, 28, 28), f"output/GAN/{epoch}.jpg", nrow=8, normalize=True)
        
        #save model
        torch.save(D, "model/GAN/discriminator.pth")
        torch.save(G, "model/GAN/generator.pth")

Epoch:     0 | D_loss: 0.31270 | G_loss: 3.10313
Epoch:     5 | D_loss: 0.36711 | G_loss: 1.49956
Epoch:    10 | D_loss: 0.56348 | G_loss: 0.74366
Epoch:    15 | D_loss: 0.31351 | G_loss: 0.98029
Epoch:    20 | D_loss: 0.17422 | G_loss: 1.30394
Epoch:    25 | D_loss: 0.47655 | G_loss: 0.92227
Epoch:    30 | D_loss: 0.46726 | G_loss: 1.21107
Epoch:    35 | D_loss: 0.40816 | G_loss: 0.87147
Epoch:    40 | D_loss: 0.61523 | G_loss: 0.81275
Epoch:    45 | D_loss: 0.30594 | G_loss: 1.04321
Epoch:    50 | D_loss: 0.38035 | G_loss: 1.30599
Epoch:    55 | D_loss: 0.59830 | G_loss: 0.94622
Epoch:    60 | D_loss: 0.37786 | G_loss: 0.83329
Epoch:    65 | D_loss: 0.50150 | G_loss: 1.07315
Epoch:    70 | D_loss: 0.61588 | G_loss: 0.58128
Epoch:    75 | D_loss: 0.65301 | G_loss: 0.61499
Epoch:    80 | D_loss: 0.50601 | G_loss: 0.78654
Epoch:    85 | D_loss: 0.68503 | G_loss: 0.56010
Epoch:    90 | D_loss: 0.61405 | G_loss: 0.65735
Epoch:    95 | D_loss: 0.52741 | G_loss: 0.78957
Epoch:   100 | D_los